In [1]:
import sys
import numpy as np
import torch as pt
import matplotlib.pyplot as plt
import sys
from pathlib import Path
sys.path.append(Path('../deterministic'))
from deterministic.mps import MPS

In [2]:
qubit_num = 10
psi = pt.rand(2**qubit_num, dtype=pt.cdouble)
mps = MPS.from_state_vector(qubit_num, psi)
print(mps)

MPS None:
	visible_num = 10
	phys_dims = [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
	bond_dims = [2, 4, 8, 16, 32, 16, 8, 4, 2]
	ext_bond_dims = [1, 2, 4, 8, 16, 32, 16, 8, 4, 2, 1]
	orth_idx = None



In [33]:
#generate a random tensor list
qubit_num = 10
bond_dim = 50
tensor_liste = [pt.rand([1, 2, bond_dim], dtype=pt.cdouble)]
for idx in range(qubit_num - 2):
    tensor_liste.append(pt.rand([bond_dim, 2, bond_dim], dtype=pt.cdouble))
tensor_liste.append(pt.rand([bond_dim, 2, 1], dtype=pt.cdouble))

In [12]:
#compute the norm of the tensor network
def norm(qubit_num, tensor_list):
    result = pt.einsum('abc,abj->cj', tensor_list[0], tensor_list[0].conj())
    for idx in range(1, qubit_num-1):
        result = pt.einsum('cj,cab->jab', result, tensor_list[idx])
        result = pt.einsum('jab,jad->bd', result, tensor_list[idx].conj())
    result = pt.einsum('ab,acj->bcj', result, tensor_list[qubit_num - 1])
    result = pt.einsum('bcj,bci->ji', result, tensor_list[qubit_num - 1].conj())
    return pt.sqrt(result)

print(norm(qubit_num=10, tensor_list=tensor_liste))

tensor([[5.5081e+11+1.5233e-05j]], dtype=torch.complex128)


In [20]:
mps = MPS.from_tensor_list(tensor_liste)
print(mps.norm())
# -> the result is the same as it should be

tensor(1.8514e+15+0.0196j, dtype=torch.complex128)


In [15]:
print(norm(10, mps.tensors))

tensor([[25.9219+0.j]], dtype=torch.complex128)


In [9]:
#make tensor specified by idx center of orthogonality
def canonicalise(tensor_list, idx, D):
    #from the left
    for index in range(0, idx):
        bond_dim_left = tensor_list[index][:,0,0].size()[0]
        bond_dim_right = tensor_list[index][0,0,:].size()[0]
        Qm, R = pt.linalg.qr(tensor_list[index].reshape(bond_dim_left*D, bond_dim_right))
        tensor_list[index] = pt.reshape(Qm, (bond_dim_left, D, Qm.size()[1]))
        tensor_list[index + 1] = pt.einsum('ab,bcd->acd', R, tensor_list[index + 1])
    for index in range(idx, len(tensor_list) - 1):
        index = len(tensor_list) - 1 - index + idx #because we want to start from the right
        bond_dim_left = tensor_list[index][:,0,0].size()[0]
        bond_dim_right = tensor_list[index][0,0,:].size()[0]
        Qm_t, R_t = pt.linalg.qr(pt.t(tensor_list[index].reshape(bond_dim_left, bond_dim_right * D)))
        Qm = pt.t(Qm_t)
        R = pt.t(R_t)
        tensor_list[index] = pt.reshape(Qm, (Qm_t.size()[1], D, bond_dim_right))
        tensor_list[index - 1] = pt.einsum('abc,cd->abd', tensor_list[index - 1], R)
    return tensor_list

print(canonicalise(tensor_list=tensor_liste, idx=9, D=2)[9])


tensor([[[-2.0688e+11-3.3123e+11j],
         [-2.0574e+11-3.2946e+11j]],

        [[-2.9711e+08+2.2473e+08j],
         [ 7.4502e+08+4.2499e+07j]]], dtype=torch.complex128)


In [4]:
mps = MPS.from_tensor_list(tensor_liste)
mps.canonicalise(3)
print(mps.tensors[3])

tensor([[[-3.0085e+11-2.4327e+11j, -3.3487e+10-2.9247e+09j,
          -6.5950e+08-1.2530e+09j,  1.9708e+09+1.0906e+09j,
          -2.6423e+07+2.1143e+07j,  8.3433e+07-2.0961e+07j,
           1.2839e+08+1.1389e+08j,  8.8659e+07-4.8502e+07j,
           1.9467e+07+2.3594e+07j,  2.5051e+06+1.0982e+07j,
          -4.0571e+06-3.7729e+06j, -5.4570e+06-7.0777e+05j,
          -1.0321e+06-1.8127e+06j,  4.3316e+06+3.3779e+06j,
           7.4226e+05+1.5740e+06j, -1.9353e+06+3.5798e+05j,
          -5.4905e+05+5.8711e+05j, -3.2113e+05-1.3669e+06j,
           7.9470e+05-8.7552e+04j, -5.9936e+05-4.6451e+05j],
         [-3.0151e+11-2.4288e+11j, -3.2174e+10-3.6360e+09j,
          -8.0982e+08-1.2474e+09j,  2.0758e+09+8.6161e+08j,
          -2.8286e+07+3.5752e+07j,  8.3377e+07-1.6083e+07j,
           1.3681e+08+1.1572e+08j,  8.8719e+07-5.2464e+07j,
           1.9488e+07+2.3712e+07j,  3.0042e+06+1.0983e+07j,
          -4.7016e+06-4.1124e+06j, -4.1050e+06-9.1517e+05j,
          -5.2864e+05-1.7988e+06j,  4.3

In [10]:
print(pt.linalg.norm(canonicalise(tensor_list=MPS.from_tensor_list(tensor_liste).tensors, idx=4, D=2)[4]))
print(norm(10, tensor_liste))
# -> the results agree as they should

tensor(5.0636e+11, dtype=torch.float64)
tensor([[5.0636e+11-6.6266e-05j]], dtype=torch.complex128)


In [7]:
def canonicalise_left_to_index(tensor_list, idx, phys_dim):
    #from the left
    for index in range(0, idx):
        bond_dim_left = tensor_list[index][:,0,0].size()[0]
        bond_dim_right = tensor_list[index][0,0,:].size()[0]
        Qm, R = pt.linalg.qr(tensor_list[index].reshape(bond_dim_left*phys_dim, bond_dim_right))
        tensor_list[index] = pt.reshape(Qm, (bond_dim_left, phys_dim, Qm.size()[1]))
        tensor_list[index + 1] = pt.einsum('ab,bcd->acd', R, tensor_list[index + 1])
    return tensor_list

In [10]:
print(canonicalise_left_to_index(tensor_liste, 9, 2)[9])

tensor([[[-2.0688e+11-3.3123e+11j],
         [-2.0574e+11-3.2946e+11j]],

        [[-2.9711e+08+2.2473e+08j],
         [ 7.4502e+08+4.2499e+07j]]], dtype=torch.complex128)


In [137]:
bits_sampled = pt.zeros(qubit_num)
probabilities_for_bits = pt.zeros(qubit_num)
# sampling algorithm
canonicalised_tensors = canonicalise_left_to_index(tensor_liste, 9, 2)
part_func = pt.einsum('ijk,ijl->kl', canonicalised_tensors[9], canonicalised_tensors[9].conj())[0,0]
result = pt.einsum('ijl,iml->jm', canonicalised_tensors[9], canonicalised_tensors[9].conj())
probs = [result[0,0,]/part_func, result[1,1]/part_func]
bits_sampled[9] = pt.multinomial(pt.tensor([probs[0].real.item(), probs[1].real.item()]), 1, replacement=True)[0].item()
probabilities_for_bits[9] = probs[int(bits_sampled[9].item())]
#now we got our first bit, we proceed with the second bit
canonicalised_tensors = canonicalise_left_to_index(tensor_liste, 8, 2)
result = pt.einsum('ik,mk->im', canonicalised_tensors[9][:,int(bits_sampled[9].item()),:], canonicalised_tensors[9][:,int(bits_sampled[9].item()),:].conj())
result = pt.einsum('im,rsi->rsm', result, canonicalised_tensors[8])
result = pt.einsum('rsm,rtm->st', result, canonicalised_tensors[8].conj())
probs = [result[0,0]/part_func/probabilities_for_bits[9], result[1,1]/part_func/probabilities_for_bits[9]]
bits_sampled[8] = pt.multinomial(pt.tensor([probs[0].real.item(), probs[1].real.item()]), 1, replacement=True)[0].item()
probabilities_for_bits[8] = probs[int(bits_sampled[8].item())]
# we got our second bit, we proceed with the third bit
canonicalised_tensors = canonicalise_left_to_index(tensor_liste, 7, 2)
result = pt.einsum('fh,jh->fj', canonicalised_tensors[9][:,int(bits_sampled[9].item()),:], canonicalised_tensors[9][:,int(bits_sampled[9].item()),:].conj())
result = pt.einsum('fj,df->dj', result, canonicalised_tensors[8][:,int(bits_sampled[8].item()),:])
result = pt.einsum('dj,lj->dl', result, canonicalised_tensors[8][:,int(bits_sampled[8].item()),:].conj())
result = pt.einsum('dl,acd->acl', result, canonicalised_tensors[7])
result = pt.einsum('acl,aml->cm', result, canonicalised_tensors[7].conj())
probs = [result[0,0]/part_func/probabilities_for_bits[9]/probabilities_for_bits[8], result[1,1]/part_func/probabilities_for_bits[9]/probabilities_for_bits[8]]
bits_sampled[7] = pt.multinomial(pt.tensor([probs[0].real.item(), probs[1].real.item()]), 1, replacement=True)[0].item()
probabilities_for_bits[7] = probs[int(bits_sampled[7].item())]
print(bits_sampled)
print(probabilities_for_bits)

tensor([0., 0., 0., 0., 0., 0., 0., 1., 0., 1.])
tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.4927, 0.4968,
        0.4886])


In [130]:
bits_sampled = pt.zeros(qubit_num)
probabilities_for_bits = pt.ones(qubit_num)
# sampling algorithm
def sampling(tensor_list, qubit_num):
    canonicalised_tensors = canonicalise_left_to_index(tensor_list, qubit_num-1, 2)
    part_func = pt.einsum('ijk,ijl->kl', canonicalised_tensors[qubit_num-1], canonicalised_tensors[qubit_num-1].conj())[0,0]

    for index in range(qubit_num):
        idx = qubit_num - 1 -index
        canonicalised_tensors = canonicalise_left_to_index(tensor_list, idx, 2)
        result = contract_part_of_mps_index_fixed(canonicalised_tensors, bits_sampled, idx, qubit_num)
        prob_for_previous_bits = pt.prod(probabilities_for_bits)
        probs = [result[0,0]/part_func/prob_for_previous_bits, result[1,1]/part_func/prob_for_previous_bits]
        bits_sampled[idx] = pt.multinomial(pt.tensor([probs[0].real.item(), probs[1].real.item()]), 1, replacement=True)[0].item()
        probabilities_for_bits[idx] = probs[int(bits_sampled[idx].item())]
    return bits_sampled, probabilities_for_bits

sampling(tensor_liste, 10)

(tensor([0., 1., 0., 0., 0., 1., 1., 1., 0., 1.]),
 tensor([0.4879, 0.5063, 0.5067, 0.5006, 0.4920, 0.5107, 0.5062, 0.4927, 0.4968,
         0.4886]))

In [121]:
def contract_part_of_mps_index_fixed(canonical_tensors, bits_sampled, idx, qubit_num):
    if idx==qubit_num-1:
        result = pt.einsum('ijl,iml->jm', canonical_tensors[idx], canonical_tensors[idx].conj())
    else:
        result = pt.einsum('fh,jh->fj', canonical_tensors[qubit_num-1][:,int(bits_sampled[qubit_num-1].item()),:], canonical_tensors[qubit_num-1][:,int(bits_sampled[qubit_num-1].item()),:].conj())
        for counter in range(qubit_num - 1 - idx - 1):
            index = qubit_num - 1 - counter - 1
            result = pt.einsum('fj,df->dj', result, canonical_tensors[index][:,int(bits_sampled[index].item()),:])
            result = pt.einsum('dj,lj->dl', result, canonical_tensors[index][:,int(bits_sampled[index].item()),:].conj())
        result = pt.einsum('rs,acr->acs', result, canonical_tensors[idx])
        result = pt.einsum('acs,ams->cm', result, canonical_tensors[idx].conj())
    return result